# Naive Bayers

In [7]:
from sklearn.model_selection import train_test_split
##from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import sklearn

In [8]:
# Função para avaliação dos modelos
def avalia_modelo(clf, X, y):
    resultados = sklearn.model_selection.cross_val_predict(clf, X, y, cv=5)
    # print( pd.crosstab(y, resultados, rownames=['Real'], colnames=['Predito'], margins=True))
    return np.mean(sklearn.model_selection.cross_val_score(clf, X, y, cv=5))

In [9]:
def doIniciaResult(tamanho):
    # Iniciar um array com vazios
    df_resultado     = [ [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ] ]
    for i in range( tamanho ):
        df_resultado.append( [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ] )

    #df_resultado = [] #pd.DataFrame(columns = colunas)
    colunas = [ 'Remetente','registros1','precisao1','registros2','precisao2','registros3','precisao3','registros4','precisao4' ]
    return df_resultado

In [10]:
def doFornecedorCut(dtNotas, fornecedores, tamanho = 1000):
    # Obtem lista de fornecedores com mais de 10000 itens
    fornecedores = dtNotas['Remetente'].value_counts()
    mask = fornecedores > tamanho
    fornecedores = list(fornecedores[mask].index)
    return fornecedores

In [11]:
# Processa o Naives
def doProcNaives(dfNFiscais, fornecedores):
  
	# Iniciar o Resutlado
	df_resultado = doIniciaResult(len( fornecedores ))

	for fonteDados in range(0, 2):
		# fonteDados = 0 --> dados com as palavras separadas manualmente
		# fonteDados = 1 --> dados sem as palavras separadas manualmente

		linha  = 0
		coluna = 0
		
		# Cria um Pipeline - Classificador Composto
		# vectorizer => transformer => classifier 
		text_clf = Pipeline([('counts' , CountVectorizer() ),
						#  ('tfidf', TfidfTransformer()),
							('classifier'  , MultinomialNB()   ),
							])

		# registra resultados
		df_resultado[linha][          0] = 'Todos'
		df_resultado[linha][ coluna + 1] = dfNFiscais.shape[0]
		df_resultado[linha][ coluna + 2] = avalia_modelo(text_clf, dfNFiscais['NovaDescricao'], dfNFiscais['Categoria'])

		print(linha,' : ',coluna+1)
		print( df_resultado[linha][          0] )
		print( df_resultado[linha][ coluna + 1] )
		print( df_resultado[linha][ coluna + 2] )
		print()

		# Cria um Pipeline - Classificador Composto
		# vectorizer => transformer => classifier 
		text_clf = Pipeline([('counts' , CountVectorizer() ),
							('tfidf', TfidfTransformer()),
							('classifier'  , MultinomialNB()   ),
							])

		# registra resultados
		df_resultado[linha][ coluna + 3] = dfNFiscais.shape[0]
		df_resultado[linha][ coluna + 4] = avalia_modelo(text_clf, dfNFiscais['NovaDescricao'], dfNFiscais['Categoria'])    

		print(linha,' : ',coluna+3)
		print( df_resultado[linha][          0] )
		print( df_resultado[linha][ coluna + 3] )
		print( df_resultado[linha][ coluna + 4] )
		print()

		# calcula precisão para fornecedores
		for fornecedor in fornecedores:
			
			linha += 1
			
			# obtem dados do fornecedor
			mask = dfNFiscais["Remetente"] == fornecedor
			dados_fornecedor = dfNFiscais[ mask ]


			# Cria um Pipeline - Classificador Composto
			# vectorizer => transformer => classifier 
			text_clf = Pipeline([('counts' , CountVectorizer() ),
							#  ('tfidf', TfidfTransformer()),
								('classifier'  , MultinomialNB()   ),
								])

			# registra resultados
			df_resultado[linha][          0] = fornecedor
			df_resultado[linha][ coluna + 1] = dados_fornecedor.shape[0]
			df_resultado[linha][ coluna + 2] = avalia_modelo(text_clf, dados_fornecedor['NovaDescricao'], 
																	dados_fornecedor['Categoria'])
			print(linha,' : ',coluna+1)
			print( df_resultado[linha][          0] )
			print( df_resultado[linha][ coluna + 1] )
			print( df_resultado[linha][ coluna + 2] )
			print()

			# Cria um Pipeline - Classificador Composto
			# vectorizer => transformer => classifier 
			text_clf = Pipeline([('counts' , CountVectorizer() ),
								('tfidf', TfidfTransformer()),
								('classifier'  , MultinomialNB()   ),
								])

			# registra resultados
			df_resultado[linha][ coluna + 3] = dados_fornecedor.shape[0]
			df_resultado[linha][ coluna + 4] = avalia_modelo(text_clf, dados_fornecedor['NovaDescricao'], 
																	dados_fornecedor['Categoria'])         
			print(linha,' : ',coluna+3)
			print( df_resultado[linha][          0] )
			print( df_resultado[linha][ coluna + 3] )
			print( df_resultado[linha][ coluna + 4] )
			print()

	return df_resultado
    

In [12]:
# create the transform
vectorizer = CountVectorizer()

text = df['NovaDescricao']
# tokenize and build vocab
vectorizer.fit(text)

# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())


NameError: name 'df' is not defined

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
palavras = list(vectorizer.vocabulary_)

In [ ]:
palavras.sort()

In [ ]:
len(palavras)

In [ ]:
palavras

In [ ]:
df_palavras = pd.DataFrame(data = palavras)


In [ ]:
df_palavras.to_csv('palavras.cvs', sep=";", index=False )

In [ ]:
vocabulário = list( vectorizer.vocabulary_ )
vocabulário

In [ ]:
vocabulário.sort()
vocabulário

In [13]:
def doNaiveBayers(dfNFiscais):    
    fornecedores = doFornecedorCut(dfNFiscais, 1000)
    resultado = doProcNaives(dfNFiscais, fornecedores)
    
    return resultado
    